In [1]:
import numpy as np
import sys
import os
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data")

In [3]:
!ls
!pwd

1.csv			 Lung12  Lung5_Rep1  Lung5_Rep3  Lung9_Rep1  vis   vis2
cosMX_showcase_meta.csv  Lung13  Lung5_Rep2  Lung6	 Lung9_Rep2  vis1  vis3
/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data


In [4]:
gene_expression = pd.read_csv('./Lung6/Lung6-Flat_files_and_images/Lung6_exprMat_file.csv')
cell_boundary = pd.read_csv('./Lung6/Lung6-Flat_files_and_images/Lung6_metadata_file.csv')
fov_position = pd.read_csv('./Lung6/Lung6-Flat_files_and_images/Lung6_fov_positions_file.csv')


In [5]:
x_coordinate = [x / 10000 for x in list(fov_position["x_global_px"])]
y_coordinate = [y / 10000 for y in list(fov_position["y_global_px"])]


In [6]:
def get_spot_fov_cellId_mapping(data_result, cell_boundary_fov_11):
    new_col_val = cell_boundary_fov_11.shape[0]* [0]
    cell_boundary_fov_11.insert(loc=0, column='spot_id', value=new_col_val)

    for i in range(cell_boundary_fov_11.shape[0]):
        one_row_sample = cell_boundary_fov_11.iloc[i]

        center_x = one_row_sample["CenterX_local_px"]
        center_y = one_row_sample["CenterY_local_px"]

        # Calculate spot_id based on CenterX_local_px and CenterY_local_px values
        # spot_id = 1 + 4 * int((center_x - 1e-6) / 1094.4) + int((center_y - 1e-6) / 912)
        spot_id = 1 + 10 * int((center_x - 1e-6) / 273.6) + int((center_y - 1e-6) / 364.8)

        # Use spot_id as needed in your code
        # print(spot_id)
        data_result = data_result.append({'spot_id' : spot_id, 'fov' : one_row_sample["fov"], 'cell_ID' : one_row_sample["cell_ID"]}, ignore_index = True)
    return data_result
    

In [7]:
data_final_result = pd.DataFrame(columns = ['spot_id', 'fov', 'cell_ID'])
fov_ids_lst = cell_boundary['fov'].unique()
print("fov_ids_lst:", fov_ids_lst)

for fov_id in tqdm(fov_ids_lst):
    # print("fov_id:", fov_id)
    cell_boundary_fov =  cell_boundary[(cell_boundary['fov']==fov_ids_lst[fov_id-1])]
    # print("cell_boundary_fov_without_spot_id:", cell_boundary_fov.shape)
    data_final_result = get_spot_fov_cellId_mapping(data_final_result, cell_boundary_fov)


fov_ids_lst: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


100%|██████████| 30/30 [01:24<00:00,  2.80s/it]


In [8]:
spot_celld_mapping = data_final_result

In [9]:
names = ['fov'] + ['spot-id=' + str(i) for i in range(1,201)]
fov_dic = {}
for i in names:
    fov_dic[i] = 0

In [10]:
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,201)]


names = ['fov'] + ['spot-id=' + str(i) for i in range(1,201)]
fov_spot_cells_stats = pd.DataFrame(columns = names)


for fov_id in fov_ids_lst:
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    
    fov_dic_sample = fov_dic
    fov_dic_sample["fov"] = fov_id
    
    for i in spot_id_lst:
        spot_id_data = fov_data[(fov_data['spot_id']==i)]
        spot_id_num = "spot-id=" + str(i)
        fov_dic_sample[spot_id_num] = spot_id_data.shape[0]
    fov_spot_cells_stats = fov_spot_cells_stats.append(fov_dic_sample, ignore_index = True)



In [11]:
spot_gene_expression = ["fov", "spot_id"]
genes_name_lst = list(gene_expression.columns)[2:]
spot_gene_expression = spot_gene_expression + genes_name_lst
spot_gene_expression = pd.DataFrame(columns = spot_gene_expression)


In [12]:
def get_spot_gene_expression(spot_cell_mapping, fov_expression, spot_id):
    genes_lst = list(fov_expression.columns)[2:]
    
    cell_id_lst = spot_cell_mapping[(spot_cell_mapping['spot_id']==spot_id)]["cell_ID"].tolist()
    
    cell_gene_expression_total = len(genes_lst)*[0]
    for cell_id in cell_id_lst:
        cell_gene_expression = fov_expression[(fov_expression['cell_ID'] == cell_id)]
        cell_gene_expression = cell_gene_expression.values.tolist()[0][2:]
        cell_gene_expression_total = np.sum([cell_gene_expression_total, cell_gene_expression], axis=0).tolist()
    
    return cell_gene_expression_total
    

In [13]:
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,201)]

for fov_id in tqdm(fov_ids_lst):
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    fov_gene_expression = gene_expression[(gene_expression['fov'] == fov_id)]
    
    for spot_id in spot_id_lst:
        to_append = [fov_id, spot_id]
        spot_gene_express = get_spot_gene_expression(fov_data, fov_gene_expression, spot_id)
        to_append = to_append + spot_gene_express
        a_series = pd.Series(to_append, index = spot_gene_expression.columns)
        spot_gene_expression = spot_gene_expression.append(a_series, ignore_index=True)    
  

100%|██████████| 30/30 [01:59<00:00,  3.99s/it]


In [14]:
CosMx_cell_type = pd.read_csv('./cosMX_showcase_meta.csv')
sample_cell_count_dic = {}
sample_fov_count_dic = {}
for i in range(1, 9):
    sample_cell_count_dic[i] = 0
    sample_fov_count_dic[i] = 0
# print("cell:", sample_cell_count_dic)
# print("fov:", sample_fov_count_dic)

sample_1_index_lst = []

cell_id_lst = CosMx_cell_type["cell_ID"].tolist()
# print("len of cell_id_lst:", len(cell_id_lst))

fov_lst = []

index = 0
for i in range(len(cell_id_lst)):
    sample_id = int(cell_id_lst[i].split("_")[1])
    fov_id = int(cell_id_lst[i].split("_")[2])
    cell_id = int(cell_id_lst[i].split("_")[3])

    if sample_id == 4:
        sample_1_index_lst.append(index)
        
    sample_cell_count_dic[sample_id] =  sample_cell_count_dic[sample_id] + 1
    if sample_id == 4:
        if fov_id not in fov_lst:
            fov_lst.append(fov_id)
    index += 1

print(sample_cell_count_dic)
print(fov_lst)
print(len(sample_1_index_lst))


{1: 98002, 2: 105800, 3: 97809, 4: 89975, 5: 87606, 6: 139504, 7: 71304, 8: 81236}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
89975


In [15]:
CosMx_cell_type_sample_1 = CosMx_cell_type.iloc[sample_1_index_lst]
CosMx_cell_type_sample_1.reset_index(drop=True, inplace=True)

for i in CosMx_cell_type_sample_1.index:
    CosMx_cell_type_sample_1.at[i,'cell_ID']= int(CosMx_cell_type_sample_1.iloc[i]["cell_ID"].split("_")[3])

CosMx_cell_type_sample_1

,Unnamed: 0,cell_ID,fov,Area,AspectRatio,Width,Height,Mean.CD298,Max.CD298,Mean.G,...,Panel,Diversity,totalcounts,log10totalcounts,background,remove_flagged_cells,patient,cell_type,niche,prop_tumor_in_100_neighbors
0,301612,4,1,2354,1.31,64,49,4252,16462,5833,...,980p,27,30,1.477121,0.006465,False,Lung6,epithelial,tumor interior,0.45
1,301613,5,1,2657,1.18,59,50,700,1542,5816,...,980p,52,69,1.838849,0.014869,False,Lung6,tumor 6,tumor interior,0.93
2,301614,7,1,2621,1.31,64,49,3399,8628,775,...,980p,35,40,1.602060,0.008619,False,Lung6,fibroblast,tumor interior,0.92
3,301615,8,1,4817,1.45,93,64,1571,3494,9873,...,980p,91,173,2.238046,0.037279,False,Lung6,tumor 6,tumor interior,0.70
4,301616,10,1,2289,0.98,55,56,1058,2088,9784,...,980p,49,75,1.875061,0.016162,False,Lung6,tumor 6,tumor interior,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89970,391582,2749,30,3036,3.58,111,31,697,1967,516,...,980p,15,34,1.531479,0.007327,False,Lung6,macrophage,macrophages,0.54
89971,391583,2752,30,1501,2.96,74,25,897,1768,19844,...,980p,29,35,1.544068,0.007542,False,Lung6,tumor 6,tumor interior,0.85
89972,391584,2753,30,1757,3.40,85,25,569,1533,1772,...,980p,29,34,1.531479,0.007327,False,Lung6,macrophage,macrophages,0.20
89973,391585,2754,30,1262,2.83,65,23,945,1535,11164,...,980p,20,24,1.380211,0.005172,False,Lung6,tumor 6,tumor interior,0.77


In [17]:
sample_1_dic = {}
for key in CosMx_cell_type_sample_1["cell_type"].tolist():
    if key not in sample_1_dic:
        sample_1_dic[key] = 1
    else:
        sample_1_dic[key] = sample_1_dic[key] + 1
# sample_1_dic


In [18]:
cell_type_lst = list(CosMx_cell_type_sample_1['cell_type'].unique())
cell_id_lst = CosMx_cell_type_sample_1["cell_ID"].tolist()


In [19]:
def get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type, cell_type_dic):
    existig_cell_ids_with_celltype = CosMx_cell_type['cell_ID'].unique()
    
    for cell_id in one_spot_cell_lst:
        if cell_id in existig_cell_ids_with_celltype:
            one_cell_sample = CosMx_cell_type[(CosMx_cell_type['cell_ID']==cell_id)]
            cell_type = one_cell_sample["cell_type"].values[0]
            cell_type_dic[cell_type] = cell_type_dic[cell_type] + 1
        else:
            print("cell id not found in groud truth!!!")
        
    return cell_type_dic


In [20]:
fov_ids_lst = spot_celld_mapping['fov'].unique()
spot_id_lst = [i for i in range(1,201)]

column_name_lst = ['fov', 'spot_id'] + cell_type_lst
ground_truth_table = pd.DataFrame(columns = column_name_lst)

for fov_id in fov_ids_lst:
    fov_data = spot_celld_mapping[(spot_celld_mapping['fov']==fov_id)]

    for spot_id in spot_id_lst:
        sample_dic = {}
        for i in column_name_lst:
            sample_dic[i] = 0
        
        spot_id_data = fov_data[(fov_data['spot_id']==spot_id)]
        one_spot_cell_lst = list(spot_id_data['cell_ID'].unique()) # all cell ids for one specific spot
        
        CosMx_cell_type_sample_fov = CosMx_cell_type_sample_1[(CosMx_cell_type_sample_1['fov']==fov_id)]
        sample_dic = get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type_sample_fov, sample_dic)
        
        sample_dic["fov"] = fov_id
        sample_dic["spot_id"] = spot_id
        ground_truth_table = ground_truth_table.append(sample_dic, ignore_index = True)




cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud t

In [21]:
ground_truth_table

,fov,spot_id,epithelial,tumor 6,fibroblast,endothelial,plasmablast,B-cell,pDC,T CD8 naive,...,T CD4 memory,Treg,T CD8 memory,mDC,tumor 12,tumor 9,neutrophil,tumor 5,tumor 13,monocyte
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
3,1.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0
4,1.0,5.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,30.0,196.0,7.0,0.0,1.0,4.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
5996,30.0,197.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
5997,30.0,198.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
5998,30.0,199.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


In [22]:
ground_truth_table.rename(columns = {'tumor 6':'tumor'}, inplace = True)
del ground_truth_table['tumor 12']
del ground_truth_table['tumor 13']
del ground_truth_table['tumor 5']
del ground_truth_table['tumor 9']

In [23]:
ground_truth_table

,fov,spot_id,epithelial,tumor,fibroblast,endothelial,plasmablast,B-cell,pDC,T CD8 naive,T CD4 naive,NK,macrophage,mast,T CD4 memory,Treg,T CD8 memory,mDC,neutrophil,monocyte
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,1.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,5.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,30.0,196.0,7.0,0.0,1.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
5996,30.0,197.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5997,30.0,198.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5998,30.0,199.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
def get_spot_x_y_range(x_global_px, y_global_px, fov_id, fov_spot_coordinates):
    fov_length = 5472
    fov_width = 3648
    x_l = x_global_px
    y_l = y_global_px
    x_h = x_l + fov_length
    y_h = y_l + fov_width
    
    for i in range(1, 21):  # 20
        for j in range(1, 11):  # 10
            spot_id = (i - 1) * 10 + j
            x = x_l + (5472 / 40.0) * 1 + (5472 / 20.0) * (i - 1)
            y = y_h - (3648 / 20.0) * 1 - (3648 / 10.0) * (j - 1)
            fov_spot_coordinates = fov_spot_coordinates.append(
                {'fov': fov_id, 'spot_id': spot_id, 'x': x, 'y': y},
                ignore_index=True
            )

    fov_spot_coordinates['x'] = fov_spot_coordinates['x']
    fov_spot_coordinates['y'] = fov_spot_coordinates['y']

    # fov_spot_coordinates['x'] = fov_spot_coordinates['x'] * 0.18 * 1e-4
    # fov_spot_coordinates['y'] = fov_spot_coordinates['y'] * 0.18 * 1e-4

    return fov_spot_coordinates



In [25]:
fov_spot_coordinates = pd.DataFrame(columns = ['fov', 'spot_id', 'x', 'y'])
for fov_id in fov_lst:
    x_px = fov_position[fov_position['fov']==fov_id]["x_global_px"].values[0]
    y_px = fov_position[fov_position['fov']==fov_id]["y_global_px"].values[0]
    fov_spot_coordinates = get_spot_x_y_range(x_px, y_px, fov_id, fov_spot_coordinates)
fov_spot_coordinates

,fov,spot_id,x,y
0,1.0,1.0,-5618.755556,-181462.177778
1,1.0,2.0,-5618.755556,-181826.977778
2,1.0,3.0,-5618.755556,-182191.777778
3,1.0,4.0,-5618.755556,-182556.577778
4,1.0,5.0,-5618.755556,-182921.377778
...,...,...,...,...
5995,30.0,196.0,21468.533333,-165036.177778
5996,30.0,197.0,21468.533333,-165400.977778
5997,30.0,198.0,21468.533333,-165765.777778
5998,30.0,199.0,21468.533333,-166130.577778


In [26]:
spot_gene_expression = spot_gene_expression.drop(columns = ["fov", "spot_id"])
ground_truth_table = ground_truth_table.drop(columns = ["fov", "spot_id"])
fov_spot_coordinates = fov_spot_coordinates.drop(columns = ["fov", "spot_id"])



import anndata as ad
st_adata = ad.AnnData(X = spot_gene_expression.values, obs = ground_truth_table, var = pd.DataFrame(index = list(spot_gene_expression.columns)), dtype=int)
st_adata.obsm["spatial"] = fov_spot_coordinates.values

spot_sums = np.sum(st_adata.X, axis=1)
mask = spot_sums > 100
filtered_data = st_adata[mask]

file_path = "/home/luqiaolin/projects/Benchmarking_paper_code/pseudo_spot_generation/cosmx_lung/Lung6.h5ad"
filtered_data.write_h5ad(file_path)





In [28]:
filtered_data.obs

,epithelial,tumor,fibroblast,endothelial,plasmablast,B-cell,pDC,T CD8 naive,T CD4 naive,NK,macrophage,mast,T CD4 memory,Treg,T CD8 memory,mDC,neutrophil,monocyte
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,7.0,0.0,1.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
5996,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5997,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5998,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
